In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import load_workbook

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

In [2]:
# Вычисление метрик качества прогнозов
def Metrics(y, y_pred):
    
    mse = round(mean_squared_error(y, y_pred), 4)
    mae = round(mean_absolute_error(y, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y, y_pred), 4)
    R2 = round(r2_score(y, y_pred), 4)
    
    return [mse, mae, mape, R2]

# Формирование таблицы метрик на тренировочной и тестовой выборке
def MetricsDF(model, x_train, x_valid, y_train, y_valid, tableName = 'Errors', testUsed = False, x_test = [], y_test = [], plot = False, savePath = ''):
    
    resDf = pd.DataFrame(index=['MSE', 'MAE', 'MAPE', 'R2'], data = 
                             {'Train': Metrics(y_train, model.predict(x_train)),
                             'Valid': Metrics(y_valid, model.predict(x_valid))})
    if testUsed:
        resDf['Test'] = Metrics(y_test, model.predict(x_test))
                        
    resDf.index.name = tableName
    display(resDf)
    
    
    if plot:
        x_valid.sort_index(inplace = True)
        y_valid.sort_index(inplace = True)
        CompareGraph(y_valid.index, y_valid, model.predict(x_valid), 'Истинные значения', 'Прогнозируемые значения', 'Сравнение прогноза с истинными данными. Valid')
        if testUsed:
            x_test.sort_index(inplace = True)
            y_test.sort_index(inplace = True)
            CompareGraph(y_test.index, y_test, model.predict(x_test), 'Истинные значения', 'Прогнозируемые значения', 'Сравнение прогноза с истинными данными. Test')
    
    # Сохранение в файл
    if savePath:
        sheet_name = f'est = {model.n_estimators}, smpl =  {model.min_samples_split}'
        
        # Номер последней строки в файле
        numRow = 0
        wb = load_workbook(savePath)
        if sheet_name in wb.sheetnames:
            numRow = wb[sheet_name].max_row + 2
        
        with pd.ExcelWriter(savePath, engine="openpyxl", mode = 'a', if_sheet_exists='overlay') as writer:
            resDf.to_excel(writer, sheet_name = sheet_name, startrow = numRow)

# Сравнение графиков двух списков значений
def CompareGraph(x, y1, y2, l1, l2, title):
    plt.figure(figsize = (10, 4))
    plt.plot(x, y1, label = l1)
    plt.plot(x, y2, label = l2)

    plt.xlabel('time')
    plt.ylabel('PM 2.5, мкг/м³')
    plt.title(title)
    
    plt.legend()
    plt.grid()
    
    plt.show()
    
# Разделение данных по сезонам
def splitDataBySeason(df, season):
    # Зима
    if season == 'winters':
        win19 = df[:'2019-03-01 00:00:00']
        win19_20 = df['2019-11-27 00:00:00':'2020-02-27 00:00:00']
        win20_21 = df['2020-11-27 00:00:00':'2021-02-25 00:00:00']
        win21_22 = df['2021-12-15 00:00:00':'2022-02-22 00:00:00']
        win22_23 = df['2022-12-13 00:00:00' : '2023-02-22 00:00:00']

        return pd.concat([win19, win19_20, win20_21, win21_22, win22_23])
    
    # Весна
    if season == 'springs':
        spr19 = pd.concat([df['2019-03-10' : '2019-05-01'], df['2019-11-01' : '2019-11-22']])
        spr20 = pd.concat([df['2020-03-01' : '2020-05-01'], df['2020-11-01' : '2020-11-10']])
        spr21 = pd.concat([df['2021-03-10' : '2021-05-01'], df['2021-11-01' : '2021-11-25']])
        spr22 = pd.concat([df['2022-03-01' : '2022-05-01'], df['2022-11-01' : '2022-11-25']])
        spr23 = df['2023-03-01' : ]

        return pd.concat([spr19, spr20, spr21, spr22, spr23])

    # Лето
    if season == 'summers':
        sum19 = df['2019-05-01' : '2019-07-12']
        sum20 = df['2020-05-01' : '2020-08-01']
        sum21 = df['2021-05-01' : '2021-08-01']
        sum22 = df['2022-05-01' : '2022-08-01']

        return pd.concat([sum19, sum20, sum21, sum22])
    
    # Осень
    if season == 'autumns':
        aut19 = df['2019-08-21' : '2019-10-16']
        aut20 = df['2020-08-17' : '2020-10-30']
        aut21 = df['2021-08-15' : '2021-11-01']
        aut22 = df['2022-08-01' : '2022-10-14']

        return pd.concat([aut19, aut20, aut21, aut22])
    

In [61]:

# Прогнозирование при помощи случайного леса
def RandForestForecast(sensor, fill, district, begin, end, season = '', valid_size = 0.3, test_index = [], params = None, gridSearch = True, plotRes = False, featImp = False,
                       path = '../data/', savePath = ''):
    
    # Данные pm 
    df = pd.read_csv(f"{path}pm25_{sensor}{fill}.csv", sep = ';', index_col = ['Date'], parse_dates = ['Date'], usecols = ['Date', district])
    df.rename(columns = {district : 'pm'}, inplace = True)

    # Исключение выборки прогнозирования
    df.drop(index = [f'2023-02-{i}' for i in range(18, 23)] + [f'2023-03-{i}' for i in range(23, 28)] +
                    [f'2022-07-{i}' for i in range(28, 32)] + [f'2022-08-01'] +
                     [f'2022-10-{i}' for i in range(10, 15)], inplace = True)
        
    
    features = ['Pressure', 'Temperature', 'Wet', 'Wind_dir', 'Wind_speed']
    for feat in features:
        df[feat] = pd.read_csv(f"{path}Features{fill}/{feat}_{sensor}.csv", sep = ';', index_col = ['Date'], parse_dates = ['Date'], usecols = ['Date', district])

        # Проверка на пропуски
        nans = pd.isnull(df[begin : end].values).sum()
             
        if nans > 0:
            print(f'Пропуски в {feat} :', nans)
 
    
    # Добавление инверсий
    inversions = pd.read_csv(f"{path}Inversions.csv", sep = ';', index_col = ['Date'], parse_dates = ['Date'])
        
    df = inversions.join(df)    
    
    
    df.dropna(inplace = True)
    
    
    testUsed = False
    # Формирование тестовой выборки
    if len(test_index):
        testUsed = True
        test_index = list(set(pd.to_datetime(test_index)) & set(df.index))
        x_test = df.loc[test_index].drop(['pm'], axis = 1)
        y_test = df.loc[test_index]['pm']
        df.drop(test_index, inplace = True)
    
    df = df[begin : end]
    
    
    tableName = f'{min(df.index.date)} — {max(df.index.date)}'
    
    
    # Срез для выбранного сезона
    if season:
        df = splitDataBySeason(df, season)
    
    # Разбиение данных
    x = df.drop(['pm'], axis = 1)
    y = df['pm']

    # train : valid выборки 1-valid_size : valid_size
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = valid_size, random_state=1)
    
    # Создаем модель
    rf = RandomForestRegressor(random_state = 100)
    
    # Поиск по сетке
    if gridSearch:
        rf = GridSearchCV(estimator = rf, param_grid = params, n_jobs = -1)
    else:
        rf.set_params(**params)
    
    # Обучаем
    rf.fit(x_train, y_train)
    
    
    if testUsed:
        MetricsDF(rf, x_train, x_valid, y_train, y_valid, tableName, testUsed, x_test, y_test, plotRes, savePath)
    else:
        MetricsDF(rf, x_train, x_valid, y_train, y_valid, tableName, testUsed, plotRes, savePath)
    
    # Определение важности признаков
    if featImp: FeatureImportance(x, rf)
    
# Важность признаков
def FeatureImportance(data, model):
    # Упорядычиваем наши фичи по значениям весов, от самой полезной к самой бесполезной
    df_importances= sorted(list(zip(data.columns, model.feature_importances_.ravel())), key=lambda tpl: tpl[1], reverse=True)

    # Создаем табличку, в которой будет показан признак и его вес
    df_importances = pd.DataFrame(df_importances, columns=['feature', 'importance']) 

    # Нумируем колонки, чтобы не путать их
    df_importances = df_importances.set_index('feature')

    # Создаем график, чтобы было нагляднее
    df_importances.plot(kind='bar', figsize=(8, 3))

    plt.title('Важность признаков')
    # Рисуем график
    plt.show()

    # Таблица с важностью признаков
    display(df_importances)

In [43]:
sensor = 's'
fill = '_fill'

district = 'Mean'

#savePath = 'Results/Results RF large periods.xlsx'

In [65]:
params = { 
    'n_estimators': np.linspace(1, 1000, 5).round().astype('int'),
    'max_features': np.linspace(0.1, 1, 5)
}
params

{'n_estimators': array([   1,  251,  500,  750, 1000]),
 'max_features': array([0.1  , 0.325, 0.55 , 0.775, 1.   ])}

## Модели, обученные за года

In [55]:
# Срез данных от begin до end
begin, end = '2020-01-01', '2023-04-06'

for season in ['winters', 'springs', 'summers', 'autumns']:
    test_index = pd.read_csv(f"../data/test_index.csv", sep = ';', dayfirst=True, parse_dates = [season])[season].values
    RandForestForecast(sensor, fill, district, begin, end, season = '', test_index = test_index, params = params, gridSearch = True, plotRes = False, featImp = False)

Пропуски в Wind_dir : 1
Пропуски в Wind_speed : 1


,Train,Valid,Test
2020-01-01 — 2023-03-22,,,
MSE,27.6447,192.4803,382.9879
MAE,3.0932,7.8409,14.4392
MAPE,0.2063,0.5375,0.4875
R2,0.9656,0.7901,0.7121


Пропуски в Wind_dir : 1
Пропуски в Wind_speed : 1


,Train,Valid,Test
2020-01-01 — 2023-03-22,,,
MSE,27.9881,244.9022,46.6978
MAE,3.0731,8.6870,5.3595
MAPE,0.2015,0.4806,0.4198
R2,0.9627,0.7953,0.5153


Пропуски в Wind_dir : 1
Пропуски в Wind_speed : 1


,Train,Valid,Test
2020-01-01 — 2023-03-22,,,
MSE,30.3169,193.2338,42.6540
MAE,3.2031,8.0789,5.0839
MAPE,0.2048,0.5464,0.6891
R2,0.9655,0.7836,0.1246


Пропуски в Wind_dir : 1
Пропуски в Wind_speed : 1


,Train,Valid,Test
2020-01-01 — 2023-03-22,,,
MSE,29.4894,194.3473,31.8114
MAE,3.1003,8.3438,4.4389
MAPE,0.1982,0.5574,0.6498
R2,0.9655,0.7950,0.2861


## Сезонные модели

In [69]:
begin, end = '2019-01-01', '2023-04-06'

for season in ['winters', 'springs', 'summers', 'autumns']:
    test_index = pd.read_csv(f"../data/test_index.csv", sep = ';', dayfirst=True, parse_dates = [season])[season].values
    RandForestForecast(sensor, fill, district, begin, end, season = season, test_index = test_index, params = params, gridSearch = True, plotRes = False, featImp = True)

Пропуски в Wind_dir : 1
Пропуски в Wind_speed : 1


,Train,Valid,Test
2020-01-01 — 2023-03-22,,,
MSE,82.1609,516.8811,434.7549
MAE,6.5167,15.0228,16.3489
MAPE,0.2331,0.4477,0.5766
R2,0.9596,0.6853,0.6732


Пропуски в Wind_dir : 1
Пропуски в Wind_speed : 1


KeyboardInterrupt: 